In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, GRU, Flatten
import pickle

In [5]:
IoT = pd.read_csv('../../data/IoT_Linux_Disk.csv')
IoT = IoT.iloc[0:500]

IoT = IoT.dropna()

# ts,PID,RDDSK,WRDSK,WCANCL,DSK,CMD,label,type

encoder=LabelEncoder()
IoT['type']=encoder.fit_transform(IoT['type'])
IoT['PID']=encoder.fit_transform(IoT['PID'])
IoT['RDDSK']=encoder.fit_transform(IoT['RDDSK'])
IoT['WRDSK']=encoder.fit_transform(IoT['WRDSK'])
IoT['WCANCL']=encoder.fit_transform(IoT['WCANCL'])
IoT['DSK']=encoder.fit_transform(IoT['DSK'])
IoT['CMD']=encoder.fit_transform(IoT['CMD'])

IoT.head()

C:\Users\spars\AppData\Local\Temp\ipykernel_10644\2859595576.py:1: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  IoT = pd.read_csv('../../data/IoT_Linux_Disk.csv')


,ts,PID,RDDSK,WRDSK,WCANCL,DSK,CMD,label,type
0,1554218915,122,16,32,6,13,48,0,0
1,1554218920,53,63,25,1,4,85,0,0
2,1554218925,95,62,29,5,4,92,0,0
3,1554218930,97,50,31,0,2,113,0,0
4,1554218935,112,53,30,0,2,133,0,0


In [6]:
x = IoT.drop(['label', 'ts', 'type'], axis=1)
y = IoT['label']

features=len(x.columns)

x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2)

In [7]:
model2 = Sequential()
    
model2.add(GRU(units = 128, activation='tanh', return_sequences=True, input_shape = (features,1)))
model2.add(Dropout(0.2))
model2.add(GRU(units = 64, activation='tanh', return_sequences=True))
model2.add(Dropout(0.2))
model2.add(Dense(1, activation='sigmoid')) 
model2.add(Flatten())
    
model2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
model2.save('../h5s/GRU.h5') 
# filename = '../h5s/GRU.h5'
# pickle.dump(model2, open(filename, 'wb'))

In [9]:
model2.fit(x_train, np.asarray(y_train).astype('float32').reshape((-1,1)), epochs = 5, batch_size=32, validation_data=(x_test, np.asarray(y_test).astype('float32').reshape((-1,1))))

Epoch 1/5
13/13 [==============================] - 4s 73ms/step - loss: 0.1207 - accuracy: 0.9225 - val_loss: 0.0174 - val_accuracy: 1.0000
Epoch 2/5
13/13 [==============================] - 0s 12ms/step - loss: 0.0123 - accuracy: 1.0000 - val_loss: 0.0070 - val_accuracy: 1.0000
Epoch 3/5
13/13 [==============================] - 0s 13ms/step - loss: 0.0052 - accuracy: 1.0000 - val_loss: 0.0036 - val_accuracy: 1.0000
Epoch 4/5
13/13 [==============================] - 0s 13ms/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0022 - val_accuracy: 1.0000
Epoch 5/5
13/13 [==============================] - 0s 13ms/step - loss: 0.0015 - accuracy: 0.9975 - val_loss: 0.0015 - val_accuracy: 1.0000


In [10]:
predictions = model2.predict(x_test)
y_pred = [round(x[0]) for x in predictions]
y_pred

4/4 [==============================] - 1s 3ms/step


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [11]:
for i in range(3):
	print(x_test.values[i].tolist(), '=>', y_pred[i], '( expected ', y_train.values[i], ')')

[56, 0, 0, 0, 0, 128] => 0 ( expected  0 )
[168, 0, 0, 0, 0, 13] => 0 ( expected  0 )
[81, 0, 0, 0, 0, 79] => 0 ( expected  0 )


In [12]:
score, acc = model2.evaluate(x_test, y_test, batch_size=1)

print('Test score:', score)
print('Test accuracy:', acc)

100/100 [==============================] - 1s 2ms/step - loss: 0.0015 - accuracy: 1.0000
Test score: 0.0015354662900790572
Test accuracy: 1.0
